In [8]:
# Scroll to the bottom for the results.

from setup import *

max_work_per_ses = 5


# read and clean data
df = pd.read_excel("Registrant_List_v0.xlsx")
df.columns = [x.strip().replace(" ","_").replace(".","_").lower() for x in df.columns]
df = df.apply(lambda x: x.str.lower() if x.dtype=='O' else x)
abs_fac = pd.factorize(df['abstract_link'])
df['abs_idx'] = abs_fac[0]

# judges breakdown
judge_idx = df['attendee_type'] == 'faculty judge'
judge = df[judge_idx]
judge_break = judge['primary_discipline'].value_counts()


# check that all abstracts have primary presenters and converesly
pres_idx = df['abs_idx'] >= 0 
nap = ~pres_idx
pp = df['presenter_type'] == 'primary presenter'
cp = df['presenter_type'] == 'co-presenter'
good = pres_idx & pp
print("There are %d abstracts with a primary presenter and %d rows without abstracts." % (good.sum(), nap.sum()))
bad = ~(good | nap)
print("There are %d bad rows" % bad.sum())
if(bad.sum()>0):
    display(df[bad],'all')


#presentation breakdown
pres = df[pres_idx]
breakdown = pres.groupby('study_category')['presentation_type'].value_counts().unstack().fillna(0).astype(int)
breakdown['judges'] = judge_break
breakdown['pres/judge'] = breakdown[['oral','poster']].max(axis=1) / breakdown['judges']
breakdown['pres/judge/session'] = breakdown['pres/judge']/4
breakdown['coverage'] = max_work_per_ses/breakdown['pres/judge/session']
cov = breakdown['coverage'].min()

print("\n")
print('The "coverage" column indicates how many times each PROJECT could be judged if we ask each judge in that category to judge %d projects in each of the 4 sessions.  Note that every category is significant above 2. '%max_work_per_ses)
display(breakdown.sort_values('pres/judge/session',ascending=False))

There are 278 abstracts with a primary presenter and 151 rows without abstracts.
There are 0 bad rows


The "coverage" column indicates how many times each PROJECT could be judged if we ask each judge in that category to judge 5 projects in each of the 4 sessions.  Note that every category is significant above 2. 


presentation_type,doc,oral,poster,judges,pres/judge,pres/judge/session,coverage
study_category,,,,,,,
physical science,1,0,23,4,5.75,1.44,3.48
engineering & computer science,7,0,34,6,5.67,1.42,3.53
business & computer information systems,1,0,15,5,3.00,0.75,6.67
education,10,0,27,10,2.70,0.68,7.41
life science,3,0,41,17,2.41,0.60,8.29
environmental science,1,0,14,6,2.33,0.58,8.57
humanities or social science,7,55,0,26,2.12,0.53,9.45
agriculture,1,0,27,14,1.93,0.48,10.37
mathematics,0,0,11,6,1.83,0.46,10.91


In [ ]:
# post_idx = pres['presentation_type'] == 'poster'
# oral_idx = pres['presentation_type'] == 'oral'
# doc_idx = pres['presentation_type'] == 'doc'
